In [ ]:
from ultralytics import YOLO
import cv2
import os
from glob import glob

In [ ]:
VEHICLE_CLASSES = {2, 3, 5, 7}
PAD = 10

vehicle_model = YOLO('../yolov8n.pt')
plate_model = YOLO('../detection-plate/runs/detect/train/weights/best.pt')

In [ ]:
def detect_plates_on_vehicles(image_path, out_path=None):
    img = cv2.imread(image_path)
    # Stage 1: detect vehicles
    v_results = vehicle_model(img)[0]
    for box in v_results.boxes:
        cls = int(box.cls)
        if cls not in VEHICLE_CLASSES:
            continue
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        # Optional: add margin to the crop
        pad = 10
        x1m, y1m = max(0, x1 - pad), max(0, y1 - pad)
        x2m, y2m = min(img.shape[1], x2 + pad), min(img.shape[0], y2 + pad)
        crop = img[y1m:y2m, x1m:x2m]

        # Stage 2: detect plates in the cropped vehicle
        p_results = plate_model(crop)[0]
        for pbox in p_results.boxes:
            px1, py1, px2, py2 = map(int, pbox.xyxy[0])
            # Map plate coords back to original image
            ox1, oy1 = px1 + x1m, py1 + y1m
            ox2, oy2 = px2 + x1m, py2 + y1m
            # Draw plate bbox
            cv2.rectangle(img, (ox1, oy1), (ox2, oy2), (0, 255, 0), 2)
            cv2.putText(
                img,
                f"plate {pbox.conf[0]:.2f}",
                (ox1, oy1 - 5),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                1,
            )

    # Draw vehicle boxes (optional)
    for box in v_results.boxes:
        cls = int(box.cls)
        if cls in VEHICLE_CLASSES:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(
                img,
                f"{vehicle_model.names[cls]} {box.conf[0]:.2f}",
                (x1, y2 + 15),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 0, 0),
                1,
            )

    if out_path:
        cv2.imwrite(out_path, img)
    return img

In [ ]:
TEST_DIR="../test/images"
TEST_RESULTS_DIR="./test-results"

os.makedirs(TEST_RESULTS_DIR, exist_ok=True)

for fname in os.listdir(TEST_DIR):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
        continue
    in_path = os.path.join(TEST_DIR, fname)
    out_path = os.path.join(TEST_RESULTS_DIR, fname)

    result = detect_plates_on_vehicles(in_path, out_path)

    out_file = os.path.join(TEST_RESULTS_DIR, os.path.basename(fname))
    cv2.imwrite(out_file, result)
    print(f"[OK] Wrote {out_path}")

In [ ]:
videos = glob("../test/videos/*.mp4")
print(videos)

In [ ]:
VIDEO_IN = videos[0]
VIDEO_OUT = videos[0].replace(".mp4", "").replace("../test/videos/", "./") + "_out.mp4"

print(VIDEO_IN)
print(VIDEO_OUT)